In [1]:
import pandas as pd, datetime as dt
import psycopg2

LocalDir =  r"############ PATH ###############"

In [2]:
def get_table_from_text(html_dir):
    
    dfs = pd.read_html(html_dir)
    
    full_text = dfs[0]["Name"][1]
    
    start = full_text.rindex('Stats by User:') + 16
    end = full_text.rindex('For any questions') - 1
    

    return full_text[start:end]

def get_report_date(html_dir):
    dfs = pd.read_html(html_dir)
    full_text = dfs[0]["Name"][1]
    return full_text[34:44]

In [3]:
tTable= get_table_from_text(LocalDir) # text of the table only

def table_headers_list(textTable):
    return [textTable[0:4], # Name
            textTable[5:16], # First login
            textTable[17:27], # Last login
            textTable[28:34], # Active 
            textTable[35:52], # No. of owned  LRs
            textTable[53:88]] # No. of days  active in past 30 days


def rest_of_table_list(textTable):
        table_as_list = textTable[89:].rsplit(' ')
        for i in table_as_list:
                i.strip()
        table_as_list.remove('')
        return table_as_list


def list_splitter(list):
    i = 0
    user_list = []
    while i <= len(list):
        user_list.append(list[i:i+6])
        i += 6
    return user_list

split_list = list_splitter(rest_of_table_list(tTable))

ValueError: No tables found

In [ ]:

def generate_nested_dict(split_list_holder):
    tab_dict = {}
    for x in split_list_holder:
        tab_dict.update({split_list_holder.index(x):x})
    tab_dict.popitem()
    return tab_dict


In [4]:
tab_df = pd.DataFrame.from_dict(data= generate_nested_dict(split_list), orient='index')

renaming_dict = {i:table_headers_list(tTable)[i] for i in range(0,len(table_headers_list(tTable)))}

tab_df.rename(columns= renaming_dict, inplace = True)
tab_df.dropna()
tab_df.replace(to_replace='None', value=None).dropna()

tab_df = tab_df.drop('First Login',axis=1)
tab_df = tab_df.drop('Last Login',axis=1)
tab_df = tab_df.drop('Active',axis=1)

#date = dt.datetime.now().strftime('%Y_%m_%d')
report_date = get_report_date(LocalDir)

for i in tab_df:
    tab_df['Report Date'] = report_date

tab_df

NameError: name 'generate_nested_dict' is not defined

In [ ]:
conn = psycopg2.connect("dbname=postgres user=postgres password=R1y2a3n4!")
cur = conn.cursor()
# use SQL regex to get each table
getTables = cur.execute("SELECT table_name FROM information_schema.tables WHERE table_name ~ '(ld_usage_report_[0-9]{4}_[0-9]{2}_[0-9]{2})'")
tableNames = cur.fetchall()

cur.close()
conn.close()

tableNames

[('ld_usage_report_2022_06_13',),
 ('ld_usage_report_2022_05_30',),
 ('ld_usage_report_2022_05_23',),
 ('ld_usage_report_2022_06_06',)]

In [ ]:
tableNames[1][0]

'ld_usage_report_2022_05_30'

In [5]:
conn = psycopg2.connect("dbname=postgres user=postgres password=########")
cur = conn.cursor()
list4Tables = []
for i in range(0,len(tableNames)):
    selectEach = cur.execute(f'SELECT * from {tableNames[i][0]}')
    list4Tables.append(cur.fetchall())
cur.close()
conn.close()

list4Tables


OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"


In [ ]:

from typing import final


dfList = []

for i in range(0,len(list4Tables)):
    anotherDataframe = pd.DataFrame(list4Tables[i])
    anotherDataframe.rename({0:'Name',1:'No. of owned LRs',2:'No. of days active in past 30 days',3:'Report Date'}, axis = 1, inplace = True)
    anotherDataframe.set_index('Name', inplace=True)
    dfList.append(anotherDataframe)

final_df = pd.concat(dfList)
dfA = final_df.groupby(['Name','Report Date']).sum('No. of owned LRs')